In [4]:
#pip install scikit-learn

In [1]:
#pip install scikit-learn --upgrade

In [1]:
#Keystroke_Manhattan_Filtered_Detector.py
 
from scipy.spatial.distance import cityblock
import numpy as np
np.set_printoptions(suppress = True)
import pandas
from sklearn import metrics
from sklearn.svm import OneClassSVM

#function to evaluate equal error rate (EER) for the detector
def evaluateEER(user_scores, imposter_scores):
    labels = [0]*len(user_scores) + [1]*len(imposter_scores)
    fpr, tpr, thresholds = metrics.roc_curve(labels, user_scores + imposter_scores)
    missrates = 1 - tpr
    farates = fpr
    dists = missrates - farates
    idx1 = np.argmin(dists[dists >= 0])
    idx2 = np.argmax(dists[dists < 0])
    x = [missrates[idx1], farates[idx1]]
    y = [missrates[idx2], farates[idx2]]
    a = ( x[0] - x[1] ) / ( y[1] - x[1] - y[0] + x[0] )
    eer = x[0] + a * ( y[0] - x[0] )
    return eer
 
class SVMDetector:
  
    def __init__(self, subjects):
        self.user_scores = []
        self.imposter_scores = []
        self.subjects = subjects
  
    def training(self):
        self.clf = OneClassSVM(kernel='rbf',gamma=26)
        self.clf.fit(self.train)
  
    def testing(self):
        self.user_scores = -self.clf.decision_function(self.test_genuine)
        self.imposter_scores = -self.clf.decision_function(self.test_imposter)
        self.user_scores = list(self.user_scores)
        self.imposter_scores = list(self.imposter_scores)
  
    def evaluate(self):
        eers = []
  
        for subject in subjects:        
            genuine_user_data = data.loc[data.subject == subject, "H.period":"H.Return"]
            imposter_data = data.loc[data.subject != subject, :]
             
            self.train = genuine_user_data[:300]
            self.train = self.train.values
            self.train = np.array(self.train)
            self.test_genuine = genuine_user_data[300:]
            self.test_genuine = self.test_genuine.values
            self.test_genuine = np.array(self.test_genuine)
            self.test_imposter = imposter_data.groupby("subject"). head(5).loc[:, "H.period":"H.Return"]
            self.test_imposter = self.test_imposter.values
            self.test_imposter = np.array(self.test_imposter)
            
            self.training()
            self.testing()
            eers.append(evaluateEER(self.user_scores, self.imposter_scores))
        return np.mean(eers)

#loading of dataset
data = pandas.read_csv("C:/Users/Sanskriti/Downloads/DSL-StrongPasswordData.csv")
subjects = data["subject"].unique()
print ("average EER for SVM detector: ")
print(SVMDetector(subjects).evaluate())

average EER for SVM detector: 
0.10818661068682427
